## Funciones de partida:

In [1]:
import networkx as nx
import itertools as it
from random import randrange
import time

def get_target_value(ejercicio, prev_value=0, max_value=10):
    if ejercicio in [1]:
        return 2
    elif ejercicio in [2,4,5,8]:
        return randrange(1,max_value)
    elif ejercicio in [3,6]:
        return 1
    elif ejercicio in [7]:
        if prev_value == 0:
            return randrange(1,max_value)
        else:
            return  prev_value + 1

def generar_matriz_y_soluciones(num_nodos, ejercicio, longitud_caminos, num_caminos=100, conectividad= 0.5, max_value_ej8=20):
    """
    Funcion que genera un grafo G aleatorio y un conjunto de caminos que constituyen una solución
    Parametros:
        num_nodos: Número de nodos del grafo a generar.
        ejercicio: numero del ejercicio para crear el grafo.
        longitud_caminos: Longitud de cada uno de los caminos que constituyen la solucion.
        num_caminos: Numero total de caminos a generar.
        conectividad: Ratio del total de nodos de G al cual un nodo estará conectado (num_nodos * conectividad)* num_nodos.
        max_value_ej8: Maximo valor a usar en el ejercicio 1.4 del boletín de prácticas.
    Devuelve:
        G: grafo generado
        adj_matrix: matriz de adyacencia a usar en los ejercicios del boletín.
        soluciones: caminos que constituyen el conjunto de solucion al ejercicio.
    """
    
    if longitud_caminos > num_nodos:
        raise ValueError(f"longitud_caminos ({longitud_caminos}) no puede ser superior num_nodos ({num_nodos}) en el grafo.")
    
    num_enlaces= (int(num_nodos*conectividad)*num_nodos)/2
    
    print("Generando matriz de adyacencia...",end="")
    t = time.time()    
    G= nx.gnm_random_graph(num_nodos, num_enlaces)
    adj_matrix= nx.to_numpy_array(G, nodelist=range(num_nodos))
    print(f"{round(time.time() - t,3)} seg.")
    
    print("Generando soluciones...",end="")
    t = time.time()    
    soluciones=[]
    i=0
    while i< num_caminos:
        p_lst= nx.generate_random_paths(G, 1, path_length=longitud_caminos-1)
        for p in p_lst:
            if len(p)==len(set(p)):
                soluciones.append(p)
                prev_value=0
                for j in range(longitud_caminos-1):
                    if ejercicio == 7:
                        value= get_target_value(ejercicio, prev_value= prev_value)
                        adj_matrix[p[j], p[j+1]]= value
                        adj_matrix[p[j+1], p[j]]= value
                        prev_value= value
                    elif ejercicio == 8:
                        value= get_target_value(ejercicio, max_value=max_value_ej8)
                        adj_matrix[p[j], p[j+1]]= value
                        adj_matrix[p[j+1], p[j]]= value
                    else:
                        value= get_target_value(ejercicio)
                        adj_matrix[p[j], p[j+1]]= value
                        adj_matrix[p[j+1], p[j]]= value
                        
                i=i+1
    
    print(f"{round(time.time() - t,3)} seg.")


    return G, adj_matrix, soluciones

Dada una matriz de adyacencia M ∈ N
p×p
en donde mi,j = 0 si no existe un camino
directo entre i y j o mi,j = d, d > 0 si existe un camino entre i y j y su distancia es d, encontrar,
dados dos puntos origen y destino (origen ̸= destino), si existe un camino de longitud ℓ que los
una de forma que la distancia total del camino no excede de un l´ımite dmax. Para este ejercicio
hacer uso de la funci´on generar matriz y soluciones pasandole como par´ametro ejercicio=5.
Ejemplo: Dada la siguiente matriz M ∈ N 5×5 de filas: a {0 0 0 1 8} , b {0 0 0 6 0}, c {0 0 0 0 9}, d {1 6 0 0 9}, e {8 0 9 9 0}
y dadas las regiones origen = b y destino = a, ℓ = 4 y dmax = 20, el camino de m´axima
seguridad que une ambas regiones es ⟨b → d → e → a⟩ cuya seguridad es 18.

In [2]:
def existe_camino(matriz, origen, destino, dmax, l):
    camino, d = encontrar_camino(matriz, origen, destino, dmax, l)
    if camino and d:
            print(f"Se encontró un camino {camino} de longitud {l} y distancia {d}, que no excede la distancia máxima {dmax}")
    else:
        print(f"No existe ningún camino que cumpla con las restricciones.")

def encontrar_camino(matriz, origen, destino, dmax, l, d_actual=0, visitados=None):
    if visitados is None:
        visitados = [False] * len(matriz)
    if origen == destino and d_actual <= dmax and l == 1:
        return [destino], d_actual 
    if d_actual > dmax or l < 1:
        return [], 0  
    visitados[origen] = True
    for i in range(len(matriz)):
        if not visitados[i] and matriz[origen][i] > 0:
            nodo, d = encontrar_camino(matriz, i, destino, dmax, l-1, d_actual + matriz[origen][i], visitados)
            if nodo:
                return [origen] + nodo, d
    visitados[origen] = False 
    return [], 0  

In [12]:
G, matriz, soluciones = generar_matriz_y_soluciones(num_nodos=10, ejercicio=5, longitud_caminos=7, num_caminos=10, conectividad=0.3)
print(G, matriz, soluciones, sep='\n')

Generando matriz de adyacencia...0.013 seg.
Generando soluciones...0.588 seg.
Graph with 10 nodes and 15 edges
[[0. 0. 3. 0. 2. 3. 0. 0. 1. 2.]
 [0. 0. 0. 7. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 8. 0. 7. 3. 0. 0.]
 [0. 7. 0. 0. 7. 9. 0. 0. 5. 0.]
 [2. 0. 8. 7. 0. 0. 0. 0. 2. 4.]
 [3. 0. 0. 9. 0. 0. 0. 0. 0. 0.]
 [0. 0. 7. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0. 2. 0.]
 [1. 0. 0. 5. 2. 0. 0. 2. 0. 0.]
 [2. 0. 0. 0. 4. 0. 0. 0. 0. 0.]]
[[1, 3, 8, 7, 2, 4, 9], [0, 4, 2, 7, 8, 3, 1], [6, 2, 0, 9, 4, 8, 7], [1, 3, 5, 0, 4, 8, 7], [7, 8, 3, 5, 0, 2, 6], [0, 2, 7, 8, 4, 3, 5], [9, 4, 8, 7, 2, 0, 5], [6, 2, 7, 8, 4, 3, 5], [6, 2, 0, 9, 4, 8, 7], [6, 2, 4, 8, 3, 5, 0]]


In [4]:
dmax = 28
l = 7
origen = 1
destino = 9
existe_camino(matriz, origen, destino, dmax, l)

Se encontró un camino [1, 5, 2, 4, 8, 6, 9] de longitud 7 y distancia 21.0, que no excede la distancia máxima 28


In [5]:
dmax = 28
l = 7
origen = 9
destino = 1
existe_camino(matriz, origen, destino, dmax, l)

Se encontró un camino [9, 6, 8, 4, 2, 5, 1] de longitud 7 y distancia 21.0, que no excede la distancia máxima 28


In [14]:
dmax = 10
l = 3
for i in range(len(matriz)):
    for j in range(len(matriz)):
        origen, destino = i, j
        print(f"Origen {i} - Destino {j}")
        existe_camino(matriz, origen, destino, dmax, l)

Origen 0 - Destino 0
No existe ningún camino que cumpla con las restricciones.
Origen 0 - Destino 1
No existe ningún camino que cumpla con las restricciones.
Origen 0 - Destino 2
Se encontró un camino [0, 4, 2] de longitud 3 y distancia 10.0, que no excede la distancia máxima 10
Origen 0 - Destino 3
Se encontró un camino [0, 4, 3] de longitud 3 y distancia 9.0, que no excede la distancia máxima 10
Origen 0 - Destino 4
Se encontró un camino [0, 8, 4] de longitud 3 y distancia 3.0, que no excede la distancia máxima 10
Origen 0 - Destino 5
No existe ningún camino que cumpla con las restricciones.
Origen 0 - Destino 6
Se encontró un camino [0, 2, 6] de longitud 3 y distancia 10.0, que no excede la distancia máxima 10
Origen 0 - Destino 7
Se encontró un camino [0, 2, 7] de longitud 3 y distancia 6.0, que no excede la distancia máxima 10
Origen 0 - Destino 8
Se encontró un camino [0, 4, 8] de longitud 3 y distancia 4.0, que no excede la distancia máxima 10
Origen 0 - Destino 9
Se encontró un

Dada una matriz de adyacencia $M \in \mathbb{N}^{p\times p}$ en donde:

- $m_{i,j} = 0$ si no existe un camino directo entre $i$ y $j$
- $m_{i,j} = 1$ si existe un camino directo entre $i$ y $j$

Se desea determinar si para desplazarse desde una región origen y destino (origen $\neq$ destino) siguiendo un camino de longitud $\ell$ se pasan por una secuencia de $r$ ($r < \ell$) regiones intermedias $S_r = \langle l \rightarrow o \rightarrow p \rangle$.

### Ejemplo

Dada la siguiente matriz $M \in \mathbb{N}^{6\times 6}$:

$M = \begin{pmatrix} 
0 & 1 & 1 & 1 & 1 & 0 \\
1 & 0 & 1 & 1 & 0 & 1 \\
1 & 1 & 0 & 0 & 1 & 1 \\
1 & 1 & 0 & 0 & 1 & 1 \\
1 & 0 & 1 & 1 & 0 & 1 \\
0 & 1 & 1 & 1 & 1 & 0
\end{pmatrix}$

Y con los siguientes valores:
- Origen: $a$
- Destino: $e$
- Longitud del camino ($\ell$): $6$
- Secuencia $S_3$: $\langle d \rightarrow b \rightarrow f \rangle$

Existe un camino que contiene dicha secuencia: $\langle a \rightarrow d \rightarrow b \rightarrow f \rightarrow c \rightarrow e \rangle$

In [7]:
def contiene_sec(camino, sec):
    if len(camino) > 2:
        check = camino.copy() 
        del check[0], check[-1]
        for j in range(len(check)):
            if check[j:j+len(sec)] == sec:
                return True
    return False

def encontrar_sec(matriz, origen, destino, l, sec, camino_actual=None):
    if camino_actual is None:
        camino_actual = []
    camino_actual.append(origen)
    caminos = []  
    if origen == destino and len(camino_actual) == l and contiene_sec(camino_actual, sec):
        caminos.append(list(camino_actual))
    if len(camino_actual) == l:
        camino_actual.pop()
        return caminos
    for i in range(len(matriz)):
        if i != origen and matriz[origen][i] > 0 and i not in camino_actual and len(caminos) < 1:
            nuevos_caminos = encontrar_sec(matriz, i, destino, l, sec, camino_actual)
            caminos.extend(nuevos_caminos)
    camino_actual.pop()
    return caminos

def existe_sec(matriz, origen, destino, l, sec):
    if l <= 2:
        print("El camino ha de ser al menos de longitud 3 para que haya nodos entre el origen y el destino")
    elif origen in sec or destino in sec:
        return print("El origen o el destino no deben de estar en la secuencia buscada")
    camino = encontrar_sec(matriz, origen, destino, l, sec)
    if camino:
        print(camino)
    else:
        print("No existe un camino que contenga la secuencia")

In [ ]:
M, m, sols = generar_matriz_y_soluciones(num_nodos=12, ejercicio=3, longitud_caminos=6, num_caminos=10, conectividad=0.6)
print(M, m, sols, sep='\n')

Generando matriz de adyacencia...3.33 seg.
Generando soluciones...0.025 seg.
Graph with 12 nodes and 42 edges
[[0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0.]
 [1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1.]
 [0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0.]]
[[1, 2, 9, 6, 4, 3], [11, 6, 5, 0, 3, 4], [6, 5, 1, 2, 9, 7], [6, 11, 10, 2, 7, 8], [3, 5, 8, 6, 9, 2], [9, 6, 4, 10, 11, 5], [4, 7, 11, 6, 8, 5], [10, 2, 1, 7, 8, 6], [5, 6, 4, 3, 0, 9], [7, 2, 3, 6, 5, 8]]


In [8]:
l = 6
sec = [2, 9, 6]
origen, destino = 1, 3
print(f"Origen {origen} - Destino {destino}")
existe_sec(m, origen, destino, l, sec)

Origen 1 - Destino 3
[[1, 2, 9, 6, 4, 3]]


In [10]:
l = 6
sec = [2, 9, 6]
for i in range(len(m)):
    for j in range(len(m)):
        origen, destino = i, j
        print(f"Origen {i} - Destino {j}")
        existe_sec(m, origen, destino, l, sec)

Origen 0 - Destino 0
No existe un camino que contenga la secuencia
Origen 0 - Destino 1
No existe un camino que contenga la secuencia
Origen 0 - Destino 2
El origen o el destino no deben de estar en la secuencia buscada
Origen 0 - Destino 3
[[0, 8, 2, 9, 6, 3]]
Origen 0 - Destino 4
[[0, 3, 2, 9, 6, 4]]
Origen 0 - Destino 5
[[0, 3, 2, 9, 6, 5]]
Origen 0 - Destino 6
El origen o el destino no deben de estar en la secuencia buscada
Origen 0 - Destino 7
[[0, 3, 2, 9, 6, 7]]
Origen 0 - Destino 8
[[0, 3, 2, 9, 6, 8]]
Origen 0 - Destino 9
El origen o el destino no deben de estar en la secuencia buscada
Origen 0 - Destino 10
[[0, 3, 2, 9, 6, 10]]
Origen 0 - Destino 11
[[0, 3, 2, 9, 6, 11]]
Origen 1 - Destino 0
[[1, 2, 9, 6, 3, 0]]
Origen 1 - Destino 1
No existe un camino que contenga la secuencia
Origen 1 - Destino 2
El origen o el destino no deben de estar en la secuencia buscada
Origen 1 - Destino 3
[[1, 2, 9, 6, 4, 3]]
Origen 1 - Destino 4
[[1, 2, 9, 6, 3, 4]]
Origen 1 - Destino 5
[[1, 2, 9,